<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/Remote_sensing_2021/blob/main/R_code_time_series.ipynb#scrollTo=WfzIAI-J6TEO">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_time_series.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Time series analysis:
> ## Greenland increase of temperature
> *data and code from Emanuela Cosma


## Install R packages
The [install.packages](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/install.packages) function is used to download and install packages from CRAN-like repositories.
To install multiple packages at the same time, you define a vector that contains the names of the packages to be installed.
The [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-10) package defines classes and methods for spatial raster data access and manipulation. The [rasterVis](https://www.rdocumentation.org/packages/rasterVis/versions/0.50.2) package complements raster providing a set of methods for enhanced visualization and interaction.
The [rgdal](https://www.rdocumentation.org/packages/rgdal/versions/1.5-23) package provides links to the [GDAL](https://gdal.org/) library (Geospatial Data Abstraction Library) and access to projection/transformation operations from the "PROJ" library.


In [ ]:
packages <- c("rgdal", "raster", "rasterVis")
install.packages(packages )

The [library](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/library) function is used to load previously installed packages.
Using the [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) function we simultaneously apply the `library()` function to all elements of the vector containing the packages.

In [ ]:
lapply(packages , library, character.only = TRUE)

Set the current working directory of the R process with the [setwd](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/getwd) function.

In [ ]:
setwd("/content")

## Upload file
1. Download Greenland surface temperature images [LST data]() for the years 2000, 2005, 2010 and 2015. Use this [dataset]() to obtain the 1979-2007 time series of the Greenland temperature.

2. On the left side of Google Colab click on the small folder icon named file ⇒ click the Upload icon ⇒ choose the files to upload from your "Downloads" folder and click OK. Otherwise, drag your files to Google Colab storage. Now your files are on Google Colab!



## Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function (RasterBrick object). 
In this case we use the [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/raster) function to read files with a single layer or band (RasterLayer object).

In [ ]:
lst_2000 <- raster("lst_2000.tif")
lst_2005 <- raster("lst_2005.tif")
lst_2010 <- raster("lst_2010.tif")
lst_2015 <- raster("lst_2015.tif")

## Visualization of spatial data
Several generic functions have been implemented for "raster" objects to create maps and other plot types. For example, use [plot](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plot) function to create a map of a raster object. Using the [par](https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/par) function it is possible to set the parameters of the graphic output and create a multi frame of images.

In [ ]:
par(mfrow=c(2,2))
plot(lst_2000)
plot(lst_2005)
plot(lst_2010)
plot(lst_2015)

The code above is somewhat redundant. We can use recursive functions like [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) to apply a certain function (in this case the `raster()` function) to a list of objects (for example our images of Greenland).
Before doing this we create a list containing our images with the [list.files](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/list.files) function.
The pattern argument allows you to select all files that have the same string part in the name.

In [ ]:
rlist <- list.files(pattern="lst")
import <- lapply(rlist, raster)

Now create a multilevel object that has a raster image for each layer or band. The new collection must be a RasterStack object with four bands one for each year. To do this we use the [stack](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/stack) function.

In [ ]:
TGr <- stack(import)
plot(TGr)

### Multi-temporal images
Individual bands can be combined into a single image by assigning individual layers or bands to one of the three color channels (red, green and blue). 
The [plotRGB](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plotRGB) function can be used to make "true (or false) color images" from multi-band satellite images.
In this case, the resulting color tone in the image will be linked to the combination of the different LST values for the years assigned to the three RGB bands.
For example, we assign the year 2005 to the red band (R = 2), the year 2010 to the green band (G = 3) and the year 2015 to the blue band (B = 4). In the image, all those areas with temperature values for 2015 that are higher than the other two time levels (2005 and 2010) will appear with shades of blue.


In [ ]:
plotRGB(TGr, r=2, g=3, b=4, stretch="Lin")

A RasterStack object can be easily represented with the [levelplot](https://www.rdocumentation.org/packages/rasterVis/versions/0.50.1/topics/levelplot-methods) function of the [rasterVis](https://www.rdocumentation.org/packages/rasterVis/versions/0.50.1) package. The `levelplot()` function allows you to create a multi-temporal frame with a subplots for each level.

In [ ]:
levelplot(TGr)

### Colour change
Use function [colorRampPalette](https://www.rdocumentation.org/packages/dichromat/versions/1.1/topics/colorRampPalette) to create a continuous color scale divided into n-levels.

In [ ]:
cl <- colorRampPalette(c("blue","light blue","pink","red"))(100)
# assign a title to each image and to the multi frame
levelplot(TGr,
          col.regions=cl, 
          # assign a title
          main = "LST variation in time",
          # rename each layer of the image
          names.attr = c("July 2000","July 2005", "July 2010", "July 2015"))

## Time series visualization
Analyze images of Greenland from 1979 to 2007.
Uses the surface temperature values of the "MODIS" product. 
We get 28 total images (one for each year) that need to be uploaded to Google Colab storage. 
Create a collection of images using the `list.files()` function and convert this list into a RasterStack object with the `stack()` function.

In [ ]:
meltlist <- list.files(pattern="melt")
melt_import <- lapply(meltlist,raster)
melt <- stack(melt_import)
# multi-temporal frame
levelplot(melt)

### Export images
The [pdf](https://www.rdocumentation.org/packages/grDevices/versions/3.6.2/topics/pdf) function starts the graphics device driver for producing PDF graphics. 
The pdf file will be named in the path (function argument) and saved on Google Colab. Click the folder icon to view all files temporarily uploaded to the machine (click the pdf name in the list to download and view the image). 
Now you can zoom in and explore each subplot better!

N.B. always use the [dev.off](https://www.rdocumentation.org/packages/imguR/versions/0.1.5/topics/dev.off) function to manage the correct closing and loading of the graphics space.

In [ ]:
pdf("/content/Melt_frame.pdf")
levelplot(melt)
dev.off()

### Residual maps
The difference between the 1979 and 2007 images highlights the temperature variations for individual pixels. 
The image shows which areas of Greenland have experienced a rise or fall in temperatures and to what extent.

In [ ]:
melt_amount <- melt$X2007annual_melt - melt$X1979annual_melt
clb <- colorRampPalette(c("blue","white","red"))(100)
# visualization melt area
plot(melt_amount, col=clb, axes = FALSE)

The `levelplot()` function displays two marginal graphs, the row and column summaries of the RasterLayer, calculated with the function defined by the FUN component of the margin list (which uses mean as the default value).

In [ ]:
levelplot(melt_amount, col.regions=clb, margin = list(FUN = 'mean'), contour=TRUE)